# Algoritmus k-mean 
V tomto cvičení vytvoříme k-mean model, který bude přiřazovat na základě měření květů kosatce ke třem druhům.

Budeme vycházet z připravených dat z minulého jupyter notebooku.

## Načtení a rozdělení dat

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
my_arrays = np.load("iris_numpy.npz")
X = my_arrays['arr_0']
y = my_arrays['arr_1']
X_norm = my_arrays['arr_2']
X_features = my_arrays['arr_3']

Protože jsme data připravovali pomocí scaler a encoderu, budeme je časem potřebovat. Proto si je načteme ze souborů.

In [ ]:
import joblib
scaler=joblib.load('classification_std_scaler.bin')
encoder=joblib.load('classification_encoder.bin')

Zobrazení mapování label na číslo druhu iris z encoderu.

In [ ]:
encoder_mapping = dict(enumerate(encoder.classes_))
encoder_mapping

Rozdělení dat na trénovací a testovací

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split (X_norm, y, test_size=0.2)

## Určení počtu k segmentů
Vstupním parametrem pro k-mean je počet segmentů, do kterých se mají data rozdělit. 

Pokud je počet segmentů součástí zadání, jako to máme my u kosatců, tak není co řešit.

Ale ukážeme si postup, jak z dat zjistit optimální počet segmentů, pokud není jasné, na kolik částí máme data rozdělit. 

K-mean algoritmus spustíme vícekrát s různým počtem segmentů. U každého výstupního modelu budeme sledovat hodnotu inertia.

inertia je součet čtvercových vzdáleností vzorků od jejich nejbližšího středu shluku vážený váhami vzorků, pokud jsou uvedeny.

In [ ]:
from sklearn.cluster import KMeans
from scipy.stats import mode
from collections import Counter

inertia_list = []
for num_clusters in range(1, 10):
    kmeans_model = KMeans(n_clusters=num_clusters, init="k-means++", n_init = 10)
    kmeans_model.fit(X_norm)
    inertia_list.append(kmeans_model.inertia_)

Optimální počet segmentů je takový, kdy se výrazně mění hodnota _inertia mezi dvěma modely s počtem segmentů rozdílným o jedna.

To zjistíme například vynesením inertia do elbow (loket) grafu. Optimální počet k je v místě, kde se graf naposledy výrazně láme.

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(range(1,10),inertia_list)
plt.scatter(range(1,10),inertia_list)
plt.scatter(3, inertia_list[3], marker="X", s=300, c="r")
plt.xlabel("Number of Clusters")
plt.ylabel("Inertia Value")
plt.title("Different Inertia Values for Different Number of Clusters")

## Trénování K-mean 
Vytvoření a natrénování k-mean modelu. 

Všimněte si, že k-mean algoritmus je model bez učitele. Při učení nepotřebuje znát správné odpovědi.

In [ ]:
kmean_model = KMeans(n_clusters=3, random_state= 2, n_init=10)
y_pred = kmean_model.fit(X_train)

Zobrazení souřadnic center shluků

In [ ]:
kmean_model.cluster_centers_

## Předpověď modelu
Spuštění modelu na trénovací a testovacích datech.

In [ ]:
y_pred_train = kmean_model.predict(X_train)
y_pred_test = kmean_model.predict(X_test)

In [ ]:
print (y_pred_train)

Implementace K-mean si id clusteru volí náhodně. Toto očíslování nemusí odpovídat hodnotě druhu kosatce, jak ho zvolil label encoder.

Pro zjištění přesnosti clusteru budeme muset sladit id clusteru, které vrací k-mean s id clusteru, které zvolil label_encoded.

V proměnné y_train_pred máme čísla k-mean clusteru.

V poli y_train máme správné odpovědi - kódování je podle encoderu.

In [ ]:
y_train

Mapování vytvoříme následovně.

Procházíme všechny tři clustery (0, 1, 2).

- ``mask = (y_train_pred == cluster)`` vytvoří logickou masku (True/False), která říká, které vzorky z trénovacích dat patří do právě procházeného clusteru.
- ``y_train[mask]`` vybere skutečné (správné) třídy jen těch vzorků, které k-means zařadil do aktuálního clusteru.
- ``mode(y_train[mask], keepdims=True).mode[0]`` najde nejčastější třídu (modus) mezi těmi vzorky. 
- ``labels_map[cluster]`` nejčastější třída se uloží do slovníku pro mapování.

In [ ]:
labels_map = {}
for cluster in range(3):
    mask = (y_pred_train == cluster)
    labels_map[cluster] = mode(y_train[mask], keepdims=True).mode[0]

Mapování id clusteru na id třídy

In [ ]:
labels_map

Kompletní mapování

In [ ]:
for id in range (0,3):
    print (f"Cluster id {id}, encoding id: {labels_map[id]}, label: {encoder_mapping[labels_map[id]]}")

Přemapování výsledku z id clusteru na správné odpovědi label_encoderu.

In [ ]:
for i in range(y_pred_train.shape[0]):
    y_pred_train[i]=labels_map[y_pred_train[i]]
    
for i in range(y_pred_test.shape[0]):
    y_pred_test[i]=labels_map[y_pred_test[i]]    

In [ ]:
y_pred_test

## Vizualizace modelu
Vytvoříme dva grafy. V jednom budou předpovědi, ve druhém skutečnost.

Grafy pro trénovací data

In [ ]:
plt.figure(figsize=(16,6))

# prediction
plt.subplot(1,2,1)
plt.scatter(X_train[y_pred_train == labels_map[0], 0], X_train[y_pred_train == labels_map[0], 1], s = 50, c = 'purple', label = 'Iris-setosa')
plt.scatter(X_train[y_pred_train == labels_map[1], 0], X_train[y_pred_train == labels_map[1], 1], s = 50, c = 'orange', label = 'Iris-versicolour')
plt.scatter(X_train[y_pred_train == labels_map[2], 0], X_train[y_pred_train == labels_map[2], 1], s = 50, c = 'green', label = 'Iris-virginica')
plt.title('Predicted Species'); plt.xlabel('petal_length'); plt.ylabel('petal_width')
# centroid
plt.scatter(kmean_model.cluster_centers_[:, 0], kmean_model.cluster_centers_[:,1], s = 100, c = 'red', label = 'Centroids')
plt.legend()

# real values
plt.subplot(1,2,2)
plt.scatter(X_train[y_train == labels_map[0], 0], X_train[y_train == labels_map[0], 1], s = 50, c = 'purple', label = 'Iris-setosa')
plt.scatter(X_train[y_train == labels_map[1], 0], X_train[y_train == labels_map[1], 1], s = 50, c = 'orange', label = 'Iris-versicolour')
plt.scatter(X_train[y_train == labels_map[2], 0], X_train[y_train == labels_map[2], 1], s = 50, c = 'green', label = 'Iris-virginica')
plt.title('True Species'); plt.xlabel('petal_length'); plt.ylabel('petal_width')

plt.legend()

Grafy pro testovací data

In [ ]:
plt.figure(figsize=(16,6))

# prediction
plt.subplot(1,2,1)
plt.scatter(X_test[y_pred_test == labels_map[0], 0], X_test[y_pred_test == labels_map[0], 1], s = 50, c = 'purple', label = 'Iris-setosa')
plt.scatter(X_test[y_pred_test == labels_map[1], 0], X_test[y_pred_test == labels_map[1], 1], s = 50, c = 'orange', label = 'Iris-versicolour')
plt.scatter(X_test[y_pred_test == labels_map[2], 0], X_test[y_pred_test == labels_map[2], 1], s = 50, c = 'green', label = 'Iris-virginica')
plt.title('Predicted Species'); plt.xlabel('petal_length'); plt.ylabel('petal_width')
# centroid
plt.scatter(kmean_model.cluster_centers_[:, 0], kmean_model.cluster_centers_[:,1], s = 100, c = 'red', label = 'Centroids')
plt.legend()

# real values
plt.subplot(1,2,2)
plt.scatter(X_test[y_test == labels_map[0], 0], X_test[y_test == labels_map[0], 1], s = 50, c = 'purple', label = 'Iris-setosa')
plt.scatter(X_test[y_test == labels_map[1], 0], X_test[y_test == labels_map[1], 1], s = 50, c = 'orange', label = 'Iris-versicolour')
plt.scatter(X_test[y_test == labels_map[2], 0], X_test[y_test == labels_map[2], 1], s = 50, c = 'green', label = 'Iris-virginica')
plt.title('True Species'); plt.xlabel('petal_length'); plt.ylabel('petal_width')
plt.legend()

## Hodnocení modelu
Jak funguje model můžeme podle různých metrik. Jakou metriku zvolit napoví zadání úlohy.

- accuracy = počet správných odpovědí / počet odpovědí
- true positive rates (recall, specifity) = TP / (TP + FN)     pacient má chorobu a byl dobře diagnostikován
- true negative rates (Sensitivity)       = TN / (TN + FP)     pacient nemá chorobu a byl dobře diagnostikován
- precision (positive predictive values)  = TP / (TP + TN)     míra přesnosti, kdy byl diagnostikován jako pozitivní
- F measure (harmonický průměr precision a recall) = 2 * (precision * recall) / (precision + recall)
- accurary = (TP + TN) / (N + P)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns

Předpovědi pro testovací data, již máme vypočítaná z vizualizace.

In [ ]:
y_pred_test

Confusion matrix

In [ ]:
cf_matrix=confusion_matrix(y_test, y_pred_test)
sns.heatmap(cf_matrix, annot=True)

Skóre modelu

In [ ]:
score=accuracy_score(y_test, y_pred_test)
print (score)

## Uložení modelu
Model jde opět uložit do souboru pro použití v inferenci.

In [ ]:
import pickle
filename = 'kmean_model.sav'
pickle.dump(kmean_model, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))

## Ladění hyperparametrů
K-mean algoritmus má různé parametry. Jedním z nich je způsob měření vzdálenosti.

Následující postup vypíše nejvhodnější kombinaci parametrů pro daná data.

Jednoduše vytvoří modely pro různé kombinace parametrů a měří jejich přesnost.

In [ ]:
from sklearn.model_selection import GridSearchCV

select_params={
               'algorithm' :["lloyd", "elkan"],
              }

grid_kmean = GridSearchCV(kmean_model, select_params, cv=5)
grid_kmean.fit(X_norm)

print('Best parameters: {}'.format(grid_kmean.best_params_))
print('Best score on training set: {}'.format(grid_kmean.best_score_))

## Vytvoření modelu ze předpřipravenou proměnnou

Vytvoříme nový model, který budou používat pouze jednu uměle vytvořenou proměnnou podle vzorce pental_width * pental_height.

Rozdělení dat na trénovací a testovací. Knihovna počítá, že X má více proměnných, proto musíme použít reshape.

In [ ]:
from sklearn.model_selection import train_test_split
X_feature_train, X_feature_test, y_feature_train, y_feature_test = train_test_split (X_features.reshape(-1,1), y, test_size=0.2)

In [ ]:
kmeanf_model = KMeans(n_clusters=3, random_state= 2, n_init=10)
kmeanf_model.fit(X_feature_train)

Nový model, nové hledání cluster id na labels.

In [ ]:
y_feature_pred_train = kmeanf_model.predict(X_feature_train)
y_feature_pred_test = kmeanf_model.predict(X_feature_test)

labels_feature_map = {}
for cluster in range(3):
    mask = (y_feature_pred_train == cluster)
    labels_feature_map[cluster] = mode(y_feature_train[mask], keepdims=True).mode[0]

In [ ]:
for id in range (0,3):
    print (f"Cluster id {id}, encoding id: {labels_feature_map[id]}, label: {encoder_mapping[labels_feature_map[id]]}")

Přemapování výsledku z id clusteru na správné odpovědi label_encoderu.

In [ ]:
for i in range(y_feature_pred_train.shape[0]):
    y_feature_pred_train[i]=labels_map[y_feature_pred_train[i]]
    
for i in range(y_feature_pred_test.shape[0]):
    y_feature_pred_test[i]=labels_map[y_feature_pred_test[i]] 

In [ ]:
cf_matrix=confusion_matrix(y_feature_test, y_feature_pred_test)
sns.heatmap(cf_matrix, annot=True)

Skóre je o něco menší než u modelu se dvěma proměnnými.

In [ ]:
accuracy_score(y_feature_test, y_feature_pred_test)

Zobrazení výsledku modelu v porovnání proti skutečnosti.

In [ ]:
X = X_feature_test 
y_true = y_feature_test
y_pred = y_feature_pred_test

plt.figure(figsize=(8, 4))
plt.scatter(X, np.zeros_like(X), c=y_true, cmap='viridis', s=100, marker='o', label='Real classes')

plt.scatter(X, np.ones_like(X)*0.1, c=y_pred, cmap='viridis', s=100, marker='x', label='Predicted clusters')

plt.yticks([]) 
plt.xlabel('Value X')
plt.legend()
plt.title('Comparison of actual classes and predicted clusters')
plt.show()